In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict

import requests
from bs4 import BeautifulSoup
import re
import os
import time
import csv
import os

if not os.path.exists('data/'):
    os.makedirs('data/')



In [4]:
# Load you personal API key
apikey = open('apikey.txt', 'r').read()

In [5]:
# Load keywords and create open-science categories
terms = pd.read_csv('keywords.csv')
categories = terms['category']
categories_unique = np.unique(np.array(categories))

In [24]:
pmcids = ['PMC5554302', 'PMC7385183', 'PMC5037725']

In [32]:
db = 'pmc'
base = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?'
dict_term = defaultdict(list)
fulfilled_categories = [0] * len(categories_unique)
for pmcid in pmcids:
    s = '{:s}db={:s}&id={:s}'.format(base, db, pmcid, apikey)
    out = requests.get(s)
    bs = BeautifulSoup(out.content, 'lxml')
    # Check if full text is available; if not - move to the next paper
    full_text_available = not (bs.findAll('sec') == [])
    if full_text_available is True:
        for i, categoryInd in enumerate (categories_unique):
            found_keyword = False
            # Loop through specific keywords related to each open-science category
            for term in terms['keyword'][terms['category'] == categoryInd]:
                for s in re.finditer(term, out.text, re.IGNORECASE):
                    fulfilled_categories[i] += 1
                    found_keyword = True

                # If one keyword is found, stop with searching for this category
                if found_keyword is True:
                    break

print(fulfilled_categories)

[0, 2, 3, 0, 0, 0, 0, 0]
